In [1]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from selenium import webdriver
from chromedriver_py import binary_path # this will get you the path variable

#svc = webdriver.ChromeService(executable_path=binary_path)

In [3]:
# ВЕРСИЯ ДЛЯ ДАННЫХ, КОГДА ТЕКСТЫ ОТФИЛЬТРОВАНЫ ПО СОДЕРЖАЩЕМУСЯ СЛОВУ
class selenium_parser():

    def __init__(self, 
                 poem_df: pd.DataFrame(),
                 base_url: str,
                 pages: int
                 ):

        self.poem_df = poem_df
        self.base_url = base_url
        self.driver = None
        self.poems_buttons = None
        self.pages = pages
    
    def get_poems_from_page(self):
        '''Из предположения, что мы уже находимся уже на определенной станице'''

        self.poems_buttons = WebDriverWait(self.driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "result-actions__context"))
        )
        #print(len( self.poems_buttons))
         
    def get_next_page(self):
        '''Переходим на следующую станицу'''

        page_button = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Вперед']"))
        )
        page_button.click()
        self.driver.execute_script("window.scrollTo(0, 0);")
        #all_handles = self.driver.window_handles
        #self.driver.switch_to.window(all_handles[0])


    def close_unnessery_page(self):
        '''Закрываем страницу, возвращаемся на базовую'''
        
        all_handles = self.driver.window_handles
        base_window_handle = all_handles[-1]
        self.driver.switch_to.window(all_handles[-1])
        self.driver.close()
        # Переключаемся обратно на базовую вкладку
        base_window_handle = all_handles[0]
        self.driver.switch_to.window(base_window_handle)
    
    def copy_text(self):
        '''Копируем текст'''
        
        all_handles = self.driver.window_handles
        # Переключаемся на последнюю (новую) вкладку
        new_window_handle = all_handles[-1]
        self.driver.switch_to.window(new_window_handle)

        element = self.driver.find_element(By.CLASS_NAME, "concordance-item")
        text_content = element.text
        print(text_content)
        self.poem_df.loc[len(self.poem_df)] = [text_content]
    
    def do_all_staff(self):
        '''Выполняем все функции'''
        time.sleep(5)
        self.driver = webdriver.Chrome()
        time.sleep(15)
        self.driver.get(self.base_url)
        time.sleep(5)
        for i in range(0, self.pages):
            time.sleep(3)
            self.get_poems_from_page()
            time.sleep(3)
            for get_poem_click in self.poems_buttons:
                time.sleep(3)
                get_poem_click.click()
                time.sleep(3)
                self.copy_text()
                time.sleep(3)
                self.close_unnessery_page()
                time.sleep(3)
                #print(self.poem_df)
            self.get_next_page()
            self.poem_df.to_csv('ya_part1.csv')
        return self.poem_df
        

In [4]:
# ВЕРСИЯ ДЛЯ ДАННЫХ, КОГДА ТЕКСТЫ ОТФИЛЬТРОВАНЫ ПО АВТОРУ
class selenium_parser():

    def __init__(self, 
                 poem_df: pd.DataFrame(),
                 base_url: str,
                 pages: int
                 ):

        self.poem_df = poem_df
        self.base_url = base_url
        self.driver = None
        self.poems_buttons = None
        self.pages = pages
    
    def get_poems_from_page(self):
        '''Из предположения, что мы уже находимся уже на определенной станице'''

        self.poems_buttons = self.driver.find_elements(By.CSS_SELECTOR, ".show-full-text[data-v-41f81f51]")
        print(len( self.poems_buttons))
         
    def get_next_page(self):
        '''Переходим на следующую станицу'''

        page_button = WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Вперед']"))
        )
        page_button.click()

        self.driver.execute_script("window.scrollTo(0, 0);")
        #all_handles = self.driver.window_handles
        #self.driver.switch_to.window(all_handles[0])


    def close_unnessery_page(self):
        '''Закрываем страницу, возвращаемся на базовую'''
        
        all_handles = self.driver.window_handles
        base_window_handle = all_handles[-1]
        self.driver.switch_to.window(all_handles[-1])
        self.driver.close()
        # Переключаемся обратно на базовую вкладку
        base_window_handle = all_handles[0]
        self.driver.switch_to.window(base_window_handle)
    
    def copy_text(self):
        '''Копируем текст'''
        
        all_handles = self.driver.window_handles
        # Переключаемся на последнюю (новую) вкладку
        new_window_handle = all_handles[-1]
        self.driver.switch_to.window(new_window_handle)

        element = self.driver.find_element(By.CLASS_NAME, "concordance-item__body")
        text_content = element.text
        print(text_content)
        self.poem_df.loc[len(self.poem_df)] = [text_content]
    
    def do_all_staff(self):
        '''Выполняем все функции'''
        time.sleep(5)
        self.driver = webdriver.Chrome()
        time.sleep(15)
        self.driver.get(self.base_url)
        time.sleep(5)
        for i in range(0, self.pages):
            time.sleep(3)
            self.get_poems_from_page()
            time.sleep(3)
            for get_poem_click in self.poems_buttons:
                
                time.sleep(3)
                get_poem_click.click()
                time.sleep(3)
                time.sleep(3)
                self.copy_text()
                time.sleep(3)
                self.close_unnessery_page()
                time.sleep(3)
                #print(self.poem_df)
            self.get_next_page()
            self.poem_df.to_csv('ya_part1.csv')
        return self.poem_df
        

In [5]:
poem_df = pd.DataFrame({'Poem Text': []})

In [6]:
#poem_df = pd.read_csv('mne_part1.csv')

In [7]:
#ya - https://ruscorpora.ru/results?search=CisqEQoJCJkCEAoYMiAKIABABXgBMgIICToBAUIPCg0KCwoDcmVxEgQKAtGPMAE=
#https://ruscorpora.ru/results?search=CjMqEQoJCLYCEAoYMiAKIABABXgBMgIICToBAUIXChUKEwoDcmVxEgwKCtC20LjQt9C90YwwAQ==

In [8]:
start_selenium = selenium_parser(poem_df, 'https://ruscorpora.ru/results?search=ClAiHAoaChgKBmF1dGhvchIOCgzQotGO0YLRh9C10LIqEAoICCQQChgyIAogAEAFeAEyAggJOgEGQhcKFQoTCgNyZXESDAoK0LLQvtC50L3QsDAB', 500)

In [9]:
start_selenium.do_all_staff()

10
С чужой стороны

(Из Гейне)

На севере мрачном, на дикой скале
Кедр одинокий под снегом белеет,
И сладко заснул он в инистой мгле,
И сон его вьюга лелеет.

Про юную пальму всё снится ему,
Что в дальных пределах Востока,
Под пламенным небом, на знойном холму
Стоит и цветет, одинока…

1823 или 1824
Саконтала

(Из Гёте)

Что юный год дает цветам ―
Их девственный румянец;
Что зрелый год дает плодам ―
Их царственный багрянец;
Что нежит взор и веселит,
Как перл, в морях цветущий;
Что греет душу и живит,
Как не́ктар, всемогущий;
Весь цвет сокровищниц мечты,
Весь полный цвет творенья,
И, словом, небо красоты
В лучах воображенья, ―
Всё, всё Поэзия слила
В тебе одной ― Саконтала́.

<1823-1824>
Друзьям при посылке "Песни радости" из Шиллера

Что пел божественный, друзья,
В порыве пламенном свободы
И в полном чувстве Бытия,
Когда на пиршество Природы
Певец, любимый сын ея,
Сзывал в единый круг народы;
И с восхищенною душей,
Во взорах ― луч животворящий,
Из чаши Гения кипящей
Он пил за здравие л

TimeoutException: Message: 


In [10]:
poem_df.to_csv('tutchev_part6.csv')

: 